# Package Imports

In [1]:
import glob
import os
import numpy as np
import pandas as pd
import sqlite3 as sql
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
sns.set(font_scale=1.2, context="paper", style="ticks")
sns.set_palette("viridis")

import mysql.connector as mariadb
from astropy.time import Time

%matplotlib inline

import plotly
plotly.offline.init_notebook_mode(connected=True)

import sys
sys.path.append("..")

In [2]:
os.environ.get("CONDA_PREFIX")

'/epyc/opt/anaconda'

In [3]:
os.nice(1)

1

In [4]:
import thor

## Run THOR

In [5]:
columnMapping = {        
        
        # Observation ID
        "obs_id" : "obs_id",
        
        # Exposure time
        "exp_mjd" : "exp_mjd",
        
        # Visit ID
        "visit_id" : "visit_id",
        
        # Field ID
        "field_id" : "field",
        
        # Field RA in degrees
        "field_RA_deg" : "fieldRA_deg",
        
        # Field Dec in degrees
        "field_Dec_deg" : "fieldDec_deg",
        
        # Night number
        "night": "nid",
        
        # RA in degrees
        "RA_deg" : "ra",
        
        # Dec in degrees
        "Dec_deg" : "decl",
        
        # Observer's x coordinate in AU
        "obs_x_au" : "HEclObsy_X_au",
        
        # Observer's y coordinate in AU
        "obs_y_au" : "HEclObsy_Y_au",
        
        # Observer's z coordinate in AU
        "obs_z_au" : "HEclObsy_Z_au",
        
        # Magnitude (UNUSED)
        "mag" : "magpsf",
        
        ### Truth Parameters
        
        # Object name
        "name" : "designation",
        
        # Observer-object distance in AU
        "Delta_au" : "Delta_au",
        
        # Sun-object distance in AU (heliocentric distance)
        "r_au" : "r_au",
        
        # Object's x coordinate in AU
        "obj_x_au" : "HEclObj_X_au",
        
        # Object's y coordinate in AU
        "obj_y_au" : "HEclObj_Y_au",
        
        # Object's z coordinate in AU
        "obj_z_au" : "HEclObj_Z_au",
        
        # Object's x velocity in AU per day
        "obj_dx/dt_au_p_day" : "HEclObj_dX/dt_au_p_day",
        
        # Object's y velocity in AU per day
        "obj_dy/dt_au_p_day" : "HEclObj_dY/dt_au_p_day",
        
        # Object's z velocity in AU per day
        "obj_dz/dt_au_p_day" : "HEclObj_dZ/dt_au_p_day",
        
        # Semi-major axis
        "a_au" : "a_au",
        
        # Inclination
        "i_deg" : "i_deg",
        
        # Eccentricity
        "e" : "e",
    }

In [6]:
simulated_ephemeris = pd.read_csv("../analysis/ztf/known_object_observations_nid_610_624.eph", sep=" ", index_col=False, low_memory=False)
orbits = pd.read_csv("../analysis/ztf/known_object_observations_nid_610_624.orb", sep=" ", index_col=False, low_memory=False)

observations = pd.read_csv("../analysis/ztf/known_object_observations_nid_610_624.txt", sep=" ", index_col=False, low_memory=False)
#observations = pd.read_csv("../analysis/ztf/observations_nid_610_624.txt", sep=" ", index_col=False, low_memory=False)

observations.sort_values(by="mjd", inplace=True)
#observations.drop(index=observations[observations["designation"].isna()].index, inplace=True)
observations.loc[observations["designation"].isna(), "designation"] = -1

In [7]:
allObjects_survey, summary_survey = thor.analyzeObservations(observations, columnMapping=columnMapping, minSamples=5, unknownIDs=[-1])

THOR: analyzeObservations
-------------------------
Analyzing observations...
Known object observations: 261369
Unknown object observations: 2731
False positive observations: 0
Percent known object observations (%): 98.966
Percent unknown object observations (%): 1.034
Percent false positive observations (%): 0.000
Unique known objects: 64313
Unique known objects with at least 5 detections: 22101

Total time in seconds: 0.6281890869140625
-------------------------



In [8]:
simulated_ephemeris["exp_mjd"] = simulated_ephemeris["mjd_utc"]

In [9]:
objs = observations[observations["nid"] == 610]["designation"].values

In [10]:
average_orbits = thor.findAverageOrbits(simulated_ephemeris[simulated_ephemeris["designation"].isin(objs)],
                                        orbits[orbits["designation"].isin(objs)], 
                                        elementType="keplerian", 
                                        dValues=[0.0, 2.0, 2.2, 2.5, 3.0, 4.0, 10])
                                        #columnMapping=columnMapping)

THOR: findAverageObject
-------------------------
Finding average orbit in 6 distance bins...
Done.
-------------------------



In [11]:
average_orbits

,orbit_id,r_au,HEclObj_dX/dt_au_p_day,HEclObj_dY/dt_au_p_day,HEclObj_dZ/dt_au_p_day,exp_mjd,RA_deg,Dec_deg,a_au,i_deg,e,designation
0,1,1.781018,0.004573,0.012181,-0.003235,58364.130486,338.510630,24.633080,1.939837,20.37268,0.086005,P3055
1,2,1.874582,0.000480,0.013408,0.000664,58364.130486,12.841012,-2.343831,2.187746,4.47398,0.153065,B4055
2,3,2.455662,0.008483,0.006678,-0.000453,58364.130486,278.003292,-15.765646,2.381806,5.74952,0.161791,n3750
3,4,2.622468,0.001155,0.010429,-0.001070,58364.130486,9.337293,18.683888,2.585887,11.49713,0.142193,K13L19W
4,5,2.754006,0.000023,0.011016,0.000197,58364.130486,17.946711,-11.988187,3.164870,12.23293,0.129994,82733
5,6,5.526367,0.005622,0.004087,0.001287,58364.130486,286.855990,4.180909,5.180951,26.47941,0.087543,85807


In [12]:
orbits["i_rad"] = np.radians(orbits["i_deg"])
orbits["ascNode_rad"] = np.radians(orbits["ascNode_deg"])
orbits["argPeri_rad"] = np.radians(orbits["argPeri_deg"])
orbits["meanAnom_rad"] = np.radians(orbits["meanAnom_deg"])

In [13]:
for orbit_id in average_orbits["orbit_id"].values:
    orbit = average_orbits[average_orbits["orbit_id"] == orbit_id]
    
    observations_orbit = observations[observations["designation"] == orbit["designation"].values[0]]
    new_epoch = observations_orbit["exp_mjd"].min()
    
    orbit_from_cat = orbits[orbits["designation"] == orbit["designation"].values[0]]
    current_epoch = orbit_from_cat["epoch_TT_mjd"]
    
    eph = thor.propagateTestParticle(orbit_from_cat[["a_au", "e", "i_rad", "ascNode_rad", "argPeri_rad", "meanAnom_rad"]].values[0], current_epoch, [new_epoch], observatoryCode="I41", mjdScale="TT", elementType="keplerian")
    
    average_orbits.loc[average_orbits["orbit_id"] == orbit_id, "r_au"] = eph["r_au"].values[0]
    average_orbits.loc[average_orbits["orbit_id"] == orbit_id, "RA_deg"] = eph["RA_deg"].values[0]
    average_orbits.loc[average_orbits["orbit_id"] == orbit_id, "Dec_deg"] = eph["Dec_deg"].values[0]
    average_orbits.loc[average_orbits["orbit_id"] == orbit_id, "exp_mjd"] = eph["mjd"].values[0]
    average_orbits.loc[average_orbits["orbit_id"] == orbit_id, "HEclObj_dX/dt_au_p_day"] = eph["HEclObj_dX/dt_au_p_day"].values[0]
    average_orbits.loc[average_orbits["orbit_id"] == orbit_id, "HEclObj_dY/dt_au_p_day"] = eph["HEclObj_dY/dt_au_p_day"].values[0]
    average_orbits.loc[average_orbits["orbit_id"] == orbit_id, "HEclObj_dZ/dt_au_p_day"] = eph["HEclObj_dZ/dt_au_p_day"].values[0]

In [14]:
average_orbits

,orbit_id,r_au,HEclObj_dX/dt_au_p_day,HEclObj_dY/dt_au_p_day,HEclObj_dZ/dt_au_p_day,exp_mjd,RA_deg,Dec_deg,a_au,i_deg,e,designation
0,1,1.781067,0.004561,0.012184,-0.003238,58364.268993,338.483061,24.595132,1.939837,20.37268,0.086005,P3055
1,2,1.874384,0.000458,0.013410,0.000666,58364.400914,12.812686,-2.355522,2.187746,4.47398,0.153065,B4055
2,3,2.455529,0.008481,0.006682,-0.000454,58364.209062,278.007204,-15.769582,2.381806,5.74952,0.161791,n3750
3,4,2.622776,0.001146,0.010428,-0.001071,58364.337951,9.307622,18.674395,2.585887,11.49713,0.142193,K13L19W
4,5,2.754025,0.000014,0.011016,0.000199,58364.379387,17.923656,-12.012809,3.164870,12.23293,0.129994,82733
5,6,5.526344,0.005622,0.004088,0.001287,58364.187870,286.853689,4.177992,5.180951,26.47941,0.087543,85807


In [15]:
average_orbits_in_observations = observations[observations["designation"].isin(average_orbits["designation"].values)].copy()
average_orbits_in_observations.drop_duplicates(subset=["designation"], keep="first", inplace=True)

In [16]:
average_orbits_in_observations

,ra,decl,field,nid,jd,exp_mjd,magpsf,sigmapsf,fid,ssnamenr,fieldRA_deg,fieldDec_deg,visit_id,mjd,HEclObsy_X_au,HEclObsy_Y_au,HEclObsy_Z_au,obs_id,designation,splitname
124101,286.853059,4.177645,488,610,2.458365e+06,58364.187870,19.5393,0.095049,2,85807,288.11944,4.55,3370,58364.187870,0.950280,-0.338662,0.000047,346585,85807,85807
12876,278.010841,-15.769092,333,610,2.458365e+06,58364.209062,19.4158,0.158122,1,493750,276.64396,-17.05,420,58364.209062,0.950400,-0.338318,0.000047,32226,n3750,493750
238093,338.492122,24.596966,645,610,2.458365e+06,58364.268993,19.1797,0.101442,1,253055,338.38480,26.15,11346,58364.268993,0.950739,-0.337342,0.000043,751567,P3055,253055
188283,9.311814,18.675340,551,610,2.458365e+06,58364.337951,20.3013,0.206710,2,2013LW19,8.62509,18.95,6314,58364.337951,0.951123,-0.336215,0.000037,536347,K13L19W,2013LW19
34851,17.927094,-12.010492,347,610,2.458365e+06,58364.379387,18.4996,0.072958,2,82733,19.29919,-9.85,636,58364.379387,0.951350,-0.335537,0.000034,72846,82733,82733
68175,12.818424,-2.352447,398,610,2.458365e+06,58364.400914,19.3790,0.119441,1,114055,15.76442,-2.65,1266,58364.400914,0.951467,-0.335185,0.000032,154874,B4055,114055


In [17]:
average_orbits

,orbit_id,r_au,HEclObj_dX/dt_au_p_day,HEclObj_dY/dt_au_p_day,HEclObj_dZ/dt_au_p_day,exp_mjd,RA_deg,Dec_deg,a_au,i_deg,e,designation
0,1,1.781067,0.004561,0.012184,-0.003238,58364.268993,338.483061,24.595132,1.939837,20.37268,0.086005,P3055
1,2,1.874384,0.000458,0.013410,0.000666,58364.400914,12.812686,-2.355522,2.187746,4.47398,0.153065,B4055
2,3,2.455529,0.008481,0.006682,-0.000454,58364.209062,278.007204,-15.769582,2.381806,5.74952,0.161791,n3750
3,4,2.622776,0.001146,0.010428,-0.001071,58364.337951,9.307622,18.674395,2.585887,11.49713,0.142193,K13L19W
4,5,2.754025,0.000014,0.011016,0.000199,58364.379387,17.923656,-12.012809,3.164870,12.23293,0.129994,82733
5,6,5.526344,0.005622,0.004088,0.001287,58364.187870,286.853689,4.177992,5.180951,26.47941,0.087543,85807


In [18]:
average_orbits = average_orbits.rename(columns={"RA_deg": columnMapping["RA_deg"], "Dec_deg": columnMapping["Dec_deg"]})

In [ ]:
allObjects_survey, summary_survey, summary_orbits = thor.runTHOR(observations, 
                                                                 average_orbits, 
                                                                 knownOrbits=orbits,
                                                                 runDir="../analysis/ztf/run_2/", 
                                                                 cellArea=500,
                                                                 cellShape="circle",
                                                                 threads=30,
                                                                 vxBins=300,
                                                                 vyBins=300,
                                                                 eps=0.005,
                                                                 minSamples=5,
                                                                 unknownIDs=[-1],
                                                                 columnMapping=columnMapping)

THOR: runTHOR
-------------------------
Running THOR with 6 orbits...

THOR: analyzeObservations
-------------------------
Analyzing observations...
Known object observations: 261369
Unknown object observations: 2731
False positive observations: 0
Percent known object observations (%): 98.966
Percent unknown object observations (%): 1.034
Percent false positive observations (%): 0.000
Unique known objects: 64313
Unique known objects with at least 5 detections: 22101

Total time in seconds: 0.4629337787628174
-------------------------

THOR: runTHOR
-------------------------
Running orbit 1...

THOR: analyzeProjections
-------------------------
Analyzing projections...
Known object observations: 5569
Unknown object observations: 286
False positive observations: 0
Percent known object observations (%): 95.115
Percent unknown object observations (%): 4.885
Percent false positive observations (%): 0.000
Unique known objects: 822
Unique known objects with at least 5 detections: 382

Total t